In [107]:
import pandas as pd
labels_df = pd.read_excel('MIT_AST_label_map.xlsx')

In [108]:
labels_df

,Unnamed: 0,label,source
0,0,Speech,human
1,1,"Male speech, man speaking",human
2,2,"Female speech, woman speaking",human
3,3,"Child speech, kid speaking",human
4,4,Conversation,human
...,...,...,...
522,522,Throbbing,background
523,523,Vibration,background
524,524,Television,background
525,525,Radio,background


In [207]:
labels_df['source'].value_counts()


source
music           146
vehicle          51
human            46
animal           36
alarm            31
work             29
material         25
mechanical       22
bird             21
background       21
explosion        19
weather          18
machine          17
foley            16
vocalization     12
door             11
insect            6
Name: count, dtype: int64

In [19]:
# extract row with label 'Whistling'

ss = labels_df['label']
print(ss)

0                             Speech
1          Male speech, man speaking
2      Female speech, woman speaking
3         Child speech, kid speaking
4                       Conversation
                   ...              
522                        Throbbing
523                        Vibration
524                       Television
525                            Radio
526                  Field recording
Name: label, Length: 527, dtype: object


In [ ]:
# add sound_class_label column and map the sound class label to the label column based on the labels_df



In [12]:
from test_mit_ast import MIT_AST_model

cfile = 'er_file_2023_09_04_7_13_08.wav'
bfile = "few_garden_files/er_file_2023_09_02_16_45_18.wav"
afile ="big_test_folder/speech_whistling2.wav"
model = MIT_AST_model()
res = model.classify(afile)
print(res)

Chirp, tweet
Whistling


In [74]:
#analyze files form folder and save results in a tuple: file name, string with labels
import os
def analyze_files_ast(folder): 
    """cretae a list of tuples with file name and string with labels
    folder: string with path to the folder with audio files
    return: list of tuples with file name and string with labels
    """   
    files = [file for file in os.listdir(folder) if file.endswith('.wav')]    
    results = []
    for file in files:
        # Construct the full path to the file
        file_path = os.path.join(folder, file)
        try:
            result = model.classify(file_path)
            if len(result) > 0:
                results.append((file_path, result))
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return results

In [14]:
test_file_path = 'big_test_folder/few_garden_files/'
res = analyze_files_ast(test_file_path)
print(res)

[('big_test_folder/few_garden_files/er_file_2023_09_04_7_49_38.wav', 'Chirp, tweet'), ('big_test_folder/few_garden_files/er_file_2023_09_02_10_00_17.wav', 'Door'), ('big_test_folder/few_garden_files/er_file_2023_09_04_7_13_08.wav', 'Owl'), ('big_test_folder/few_garden_files/er_file_2023_09_02_16_45_18.wav', 'Caw')]


In [70]:
from tqdm import tqdm

# method to add a column with 'MIT_AST_label' to the dataframe, allign by file name
def add_label_column(df, results):
    """
    Add a column with 'MIT_AST_label' to the dataframe, allign by file name
    df: dataframe with metadata
    results: list of tuples with file name and string with labels
    return: dataframe with 'MIT_AST_label' column
    """
    # create column  if the column not already exists
    #if 'MIT_AST_label' not in df.columns:
    df['MIT_AST_label'] = None
    for file, label in results:
        df.loc[df['filename'] == file, 'MIT_AST_label'] = label
    return df

# get metadata_file and output_file from folder name
def get_metadata_output_files(folder):
    """
    Get metadata_file and output_file from folder name
    folder: string with folder name
    return: metadata_file, output_file
    """
    metadata_file = "metadata/" +folder[:-1] + "_metadata_birdnet_labeled.xlsx"
    output_file = metadata_file.split(".")[0] + "_mit_ast.xlsx"
    return metadata_file, output_file



folder = "garden_16032024/"
metadata_file, output_file = get_metadata_output_files(folder)
metadata_file, output_file

('metadata/garden_16032024_metadata_birdnet_labeled.xlsx',
 'metadata/garden_16032024_metadata_birdnet_labeled_mit_ast.xlsx')

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

def analyze_folders(folder, batch_size=10):
    """
    Analyze all files in the folder of folders, processing subfolders in batches.
    Each batch's results are saved to a CSV file as a backup.
    The function returns the combined results from all batches.
    
    Args:
    folder: string with path to the folder containing subfolders.
    batch_size: number of subfolders to process in each batch (optional, default is 10).
    
    Returns:
    results: list of tuples with file name and string with labels from all subfolders.
    """
    # List all relevant subfolders, excluding hidden ones and ensuring they are directories
    subfolders = [f for f in os.listdir(folder) if not f.startswith('.') and os.path.isdir(os.path.join(folder, f))]
    total_results = []
    csv_backup_path = os.path.join(folder, 'batch_results_backup.csv')
    
    # Process subfolders in batches
    for i in tqdm(range(0, len(subfolders), batch_size)):
        batch_folders = subfolders[i:i + batch_size]
        batch_results = []

        for f in batch_folders:
            print(f"Analyzing folder {f}")
            subfolder_path = os.path.join(folder, f)
            batch_results += analyze_files_ast(subfolder_path + "/")
        
        # Convert batch results to DataFrame and save to CSV
        df_batch = pd.DataFrame(batch_results, columns=['Filename', 'Label'])
        if os.path.exists(csv_backup_path):
            df_batch.to_csv(csv_backup_path, mode='a', header=False, index=False)
        else:
            df_batch.to_csv(csv_backup_path, mode='w', header=True, index=False)
        
        # Accumulate results from each batch
        total_results.extend(batch_results)

    return total_results



In [112]:
f = pd.read_excel("metadata/garden_19102023_metadata_birdnet_labeled_mit_ast.xlsx")
#f.loc[f['filename'] == res[0][0], 'MIT_AST_label'] = res[0][1]
f['filename'][1]

'garden_19102023/160/er_file_2023_10_19_9_38_33.wav'

In [80]:
def pipeline_mit_ast(folder, batch_size=10):
    """
    Run the whole pipeline for the folder
    folder: string with folder name
    """
    metadata_file, output_file = get_metadata_output_files(folder)
    results = analyze_folders(folder,batch_size)
    df = pd.read_excel(metadata_file)
    df = add_label_column(df, results)
    df.to_excel(output_file)

In [89]:
# test the pipeline
folder = "garden_08032024/"

pipeline_mit_ast(folder)


100%|██████████| 26/26 [3:32:34<00:00, 490.54s/it]  


In [90]:
mf = pd.read_excel("metadata/garden_08032024_metadata_birdnet_labeled_mit_ast.xlsx")
mf['MIT_AST_label'].notnull().sum()


25805

In [91]:
# readbackup file
import pandas as pd
df = pd.read_csv('garden_08032024/batch_results_backup.csv')
df.shape

(25809, 2)

In [145]:
# concat 2 excel files
import pandas as pd
df1 = pd.read_excel('metadata/garden_03032024_metadata_birdnet_labeled_mit_ast.xlsx')
df2 = pd.read_excel('metadata/garden_08032024_metadata_birdnet_labeled_mit_ast.xlsx')
df3 = pd.read_excel('metadata/garden_16032024_metadata_birdnet_labeled_mit_ast.xlsx')
# check if the files have the same columns
# change 'label' to 'BirdNET'
df1.rename(columns={'label': 'BirdNET'}, inplace=True)
df2.rename(columns={'label': 'BirdNET'}, inplace=True)
df3.rename(columns={'label': 'BirdNET'}, inplace=True)

#save back to excel
df1.to_excel('metadata/garden_03032024_metadata_birdnet_labeled_mit_ast.xlsx')
df2.to_excel('metadata/garden_08032024_metadata_birdnet_labeled_mit_ast.xlsx')
df3.to_excel('metadata/garden_16032024_metadata_birdnet_labeled_mit_ast.xlsx')

new_df = pd.concat([df1, df2, df3], ignore_index=True)
new_df.head()




,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,...,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3
0,0,0,0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,...,286,0,36.0,5.462251,51.417782,NaN,Bird,NaN,NaN,NaN
1,1,1,1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,...,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,NaN,NaN,NaN
2,2,2,2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,...,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,NaN,NaN,NaN
3,3,3,3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,...,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,NaN,NaN,NaN
4,4,4,4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,...,286,0,36.0,5.462251,51.417782,NaN,Bird,NaN,NaN,NaN


In [160]:
# drop all columns named 'Unnamed'
new_df = new_df.loc[:, ~new_df.columns.str.contains('^Unnamed')]



In [161]:
new_df.shape

(59039, 25)

In [162]:
# Merging df_con with labels_df
# Example of merging with specific column names handling
new_df = new_df.merge(labels_df, left_on='MIT_AST_label', right_on='label', how='left')
new_df.rename(columns={'source': 'sound_class_label'}, inplace=True)
new_df.drop(columns=['label'], inplace=True)  # Assuming 'label_y' is the extra column from labels_new_df
new_df.head()

,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,Unnamed: 0,sound_class_label
0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,20240303,...,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,111,bird
1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,20240303,...,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,119,bird
2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,20240303,...,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,119,bird
3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,20240303,...,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,111,bird
4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,20240303,...,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,111,bird


In [163]:
new_df = new_df.loc[:, ~new_df.columns.str.contains('^Unnamed')]

new_df.head()

,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,bird
1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,bird


In [168]:
df_concat = pd.read_excel('metadata/garden_concat_metadata_birdnet_labeled_mit_ast_.xlsx')
df_concat.shape

(347650, 29)

In [170]:
df_concat.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,0.0,0,0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves,NaN
1,1.0,1,1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur,NaN
2,2.0,2,2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock,NaN
3,3.0,3,3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN
4,4.0,4,4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,...,9.1,98.0,193.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN


,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,0.216242,0.50,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves,NaN
1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,0.662231,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur,NaN
2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,0.062379,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock,NaN
3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,0.063947,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN
4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,0.030672,0.25,20231216,...,9.1,98.0,193.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN


In [171]:
# Example of merging with specific column names handling
df_concat = df_concat.merge(labels_df, left_on='MIT_AST_label', right_on='label', how='left')
df_concat.rename(columns={'source': 'sound_class_label'}, inplace=True)
df_concat.drop(columns=['label'], inplace=True)  # Assuming 'label_y' is the extra column from labels_df_concat
df_concat.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0_x,sessionId,time,filename,th1,th1_value,th2,th2_value,...,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label,Unnamed: 0_y,sound_class_label
0,0.0,0,0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,...,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves,NaN,284.0,weather
1,1.0,1,1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,...,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur,NaN,65.0,foley
2,2.0,2,2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,...,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock,NaN,359.0,door
3,3.0,3,3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,...,204.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN,354.0,door
4,4.0,4,4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,...,193.0,0.0,0.0,5.462251,51.417782,NaN,Door,NaN,354.0,door


In [173]:
df_concat.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0_x', 'sessionId', 'time',
       'filename', 'th1', 'th1_value', 'th2', 'th2_value', 'th3', 'th3_value',
       'wudate', 'datetime', 'precipRate', 'pressureMax', 'dewptAvg',
       'windgustHigh', 'windspeedAvg', 'tempAve', 'humidityAvg', 'winddirAvg',
       'uvHigh', 'solarRadiationHigh', 'lon', 'lat', 'BirdNET',
       'MIT_AST_label', 'sound_class_label', 'Unnamed: 0_y',
       'sound_class_label'],
      dtype='object')

In [175]:
# Drop the second 'sound_class_label' column by selecting all columns except the second occurrence
df_concat = df_concat.loc[:, ~df_concat.columns.duplicated(keep='last')]  # This keeps the first occurrence and drops subsequent duplicates
df_concat.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0_x,sessionId,time,filename,th1,th1_value,th2,th2_value,...,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,Unnamed: 0_y,sound_class_label
0,0.0,0,0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,...,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves,284.0,weather
1,1.0,1,1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,...,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur,65.0,foley
2,2.0,2,2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,...,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock,359.0,door
3,3.0,3,3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,...,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Door,354.0,door
4,4.0,4,4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,...,98.0,193.0,0.0,0.0,5.462251,51.417782,NaN,Door,354.0,door


In [176]:
df_concat = df_concat.loc[:, ~df_concat.columns.str.contains('^Unnamed')]

df_concat.head()

,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,0.216242,0.50,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves,weather
1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,0.662231,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur,foley
2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,0.062379,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock,door
3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,0.063947,0.25,20231216,...,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Door,door
4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,0.030672,0.25,20231216,...,9.1,98.0,193.0,0.0,0.0,5.462251,51.417782,NaN,Door,door


In [177]:
new_df.head()

,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,bird
1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,20240303,...,9.6,88.0,286,0,36.0,5.462251,51.417782,NaN,Bird,bird


In [178]:
df_con = pd.concat([new_df, df_concat])
df_con.shape

(406689, 26)

In [179]:
df_con.head()

,sessionId,time,filename,th1,th1_value,th2,th2_value,th3,th3_value,wudate,...,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,BirdNET,MIT_AST_label,sound_class_label
0,test,_2024_03_03_8_34_48,garden_03032024/0/er_file_2024_03_03_8_34_48.wav,0.382969,0.50,0.790168,0.50,0.255452,0.50,20240303,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird
1,test,_2024_03_03_8_34_50,garden_03032024/0/er_file_2024_03_03_8_34_50.wav,0.176002,0.25,0.932867,0.50,0.271969,0.50,20240303,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,test,_2024_03_03_8_34_53,garden_03032024/0/er_file_2024_03_03_8_34_53.wav,0.278563,0.25,0.474437,0.50,0.023406,0.24,20240303,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,test,_2024_03_03_8_34_56,garden_03032024/0/er_file_2024_03_03_8_34_56.wav,0.263044,0.25,0.812372,0.64,0.172581,0.24,20240303,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,test,_2024_03_03_8_34_59,garden_03032024/0/er_file_2024_03_03_8_34_59.wav,0.164429,0.25,0.871671,0.64,0.044865,0.24,20240303,...,9.6,88.0,286.0,0.0,36.0,5.462251,51.417782,NaN,Bird,bird


In [180]:
df_con.to_excel('metadata/garden_concat_metadata_birdnet_labeled_mit_ast_24.04.xlsx')

In [181]:
columns = df_con.columns
columns

Index(['sessionId', 'time', 'filename', 'th1', 'th1_value', 'th2', 'th2_value',
       'th3', 'th3_value', 'wudate', 'datetime', 'precipRate', 'pressureMax',
       'dewptAvg', 'windgustHigh', 'windspeedAvg', 'tempAve', 'humidityAvg',
       'winddirAvg', 'uvHigh', 'solarRadiationHigh', 'lon', 'lat', 'BirdNET',
       'MIT_AST_label', 'sound_class_label'],
      dtype='object')

In [184]:
# leave only 'datetime', 'precipRate', 'pressureMax','dewptAvg', 'windgustHigh', 'windspeedAvg', 'tempAve', 'humidityAvg','winddirAvg', 'uvHigh', 'solarRadiationHigh', 'lon', 'lat', 'BirdNET','MIT_AST_label', 'sound_class_label'
columns_to_keep = ['datetime', 'precipRate', 'pressureMax','dewptAvg', 'windgustHigh', 'windspeedAvg', 'tempAve', 'humidityAvg','winddirAvg', 'uvHigh', 'solarRadiationHigh', 'BirdNET','MIT_AST_label', 'sound_class_label']
df_stats = df_con[columns_to_keep]
df_stats.head()

,datetime,precipRate,pressureMax,dewptAvg,windgustHigh,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,BirdNET,MIT_AST_label,sound_class_label
0,2024-03-03 08:34:48,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,NaN,Bird,bird
1,2024-03-03 08:34:50,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,2024-03-03 08:34:53,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,2024-03-03 08:34:56,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,2024-03-03 08:34:59,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,NaN,Bird,bird


In [186]:
df_stats.describe()

,datetime,precipRate,pressureMax,dewptAvg,windgustHigh,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh
count,406689,404553.000000,406689.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000
mean,2023-12-23 00:16:55.408518400,0.357925,1008.954810,8.598520,12.481722,4.208838,10.869927,86.964778,193.873915,0.447603,67.606959
min,2023-08-11 17:59:32,0.000000,979.410000,-9.900000,0.000000,0.000000,-6.500000,40.700000,0.000000,0.000000,0.000000
25%,2023-11-19 12:59:53,0.000000,1000.340000,6.500000,7.200000,1.800000,8.100000,81.200000,167.000000,0.000000,0.000000
50%,2023-12-23 20:31:10,0.000000,1010.330000,8.600000,11.100000,3.900000,10.400000,89.900000,199.000000,0.000000,5.100000
75%,2024-01-31 08:23:56,0.000000,1017.340000,10.700000,16.600000,6.200000,12.200000,96.100000,215.000000,0.000000,74.100000
max,2024-03-23 05:53:04,39.600000,1034.640000,22.800000,43.900000,17.300000,32.800000,99.000000,357.000000,13.000000,1351.700000
std,NaN,1.358902,11.791507,4.895714,7.874728,2.995714,5.702933,11.493153,40.898593,1.115351,126.441388


In [187]:
#check for missing values
df_stats.isnull().sum()



datetime                   0
precipRate              2136
pressureMax                0
dewptAvg                2136
windgustHigh            2136
windspeedAvg            2136
tempAve                 2136
humidityAvg             2136
winddirAvg              2136
uvHigh                  2136
solarRadiationHigh      2136
BirdNET               334854
MIT_AST_label          16564
sound_class_label      16564
dtype: int64

In [189]:
#drop 2136 rows with missing values in preipRate
ststs_full = df_stats.dropna(subset=['precipRate'], inplace=False)
ststs_full.isnull().sum()

datetime                   0
precipRate                 0
pressureMax                0
dewptAvg                   0
windgustHigh               0
windspeedAvg               0
tempAve                    0
humidityAvg                0
winddirAvg                 0
uvHigh                     0
solarRadiationHigh         0
BirdNET               333023
MIT_AST_label          16564
sound_class_label      16564
dtype: int64

In [196]:
ststs_full.shape

(404553, 14)

In [190]:
ststs_full.describe()# DF from alll data without nan values in weather rows

,datetime,precipRate,pressureMax,dewptAvg,windgustHigh,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh
count,404553,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000,404553.000000
mean,2023-12-23 11:15:45.417939968,0.357925,1008.901972,8.598520,12.481722,4.208838,10.869927,86.964778,193.873915,0.447603,67.606959
min,2023-08-11 17:59:32,0.000000,979.410000,-9.900000,0.000000,0.000000,-6.500000,40.700000,0.000000,0.000000,0.000000
25%,2023-11-19 18:26:35,0.000000,1000.240000,6.500000,7.200000,1.800000,8.100000,81.200000,167.000000,0.000000,0.000000
50%,2023-12-23 23:04:09,0.000000,1010.230000,8.600000,11.100000,3.900000,10.400000,89.900000,199.000000,0.000000,5.100000
75%,2024-01-31 10:52:43,0.000000,1017.240000,10.700000,16.600000,6.200000,12.200000,96.100000,215.000000,0.000000,74.100000
max,2024-03-23 05:53:04,39.600000,1034.640000,22.800000,43.900000,17.300000,32.800000,99.000000,357.000000,13.000000,1351.700000
std,NaN,1.358902,11.783670,4.895714,7.874728,2.995714,5.702933,11.493153,40.898593,1.115351,126.441388


In [193]:
#save ststs_full to csv
ststs_full.to_csv('metadata/stats_full_25.04_full.csv', index=False)

In [198]:
ststs_full.head()

,datetime,precipRate,pressureMax,dewptAvg,windgustHigh,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,BirdNET,MIT_AST_label,sound_class_label
0,2024-03-03 08:34:48,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,NaN,Bird,bird
1,2024-03-03 08:34:50,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
2,2024-03-03 08:34:53,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Owl,bird
3,2024-03-03 08:34:56,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,Streptopelia decaocto_Eurasian Collared-Dove,Bird,bird
4,2024-03-03 08:34:59,0.0,995.33,7.6,3.5,1.4,9.6,88.0,286.0,0.0,36.0,NaN,Bird,bird


In [202]:
ststs_full['BirdNET'].value_counts()[:10]

BirdNET
Corvus monedula_Eurasian Jackdaw                24851
Parus major_Great Tit                           14994
Cyanistes caeruleus_Eurasian Blue Tit            7958
Streptopelia decaocto_Eurasian Collared-Dove     3819
Columba palumbus_Common Wood-Pigeon              3182
Passer domesticus_House Sparrow                  2359
Erithacus rubecula_European Robin                1946
Prunella modularis_Dunnock                       1740
Turdus merula_Eurasian Blackbird                 1729
Corvus corone_Carrion Crow                       1562
Name: count, dtype: int64

In [204]:
ststs_full['MIT_AST_label'].value_counts()[:20]

MIT_AST_label
Rustling leaves                            52368
Heart sounds, heartbeat                    36823
Bird                                       35235
Speech                                     24995
Vehicle                                    24704
Animal                                     24393
Raindrop                                   21468
Chirp, tweet                               10980
Clip-clop                                  10916
Caw                                         9345
Owl                                         9341
Wind noise (microphone)                     8838
Heart murmur                                8199
Silence                                     7876
Rain on surface                             7674
Wind                                        7454
Door                                        7223
Bird vocalization, bird call, bird song     6734
Crow                                        6476
Fowl                                        5332
Name: 